Лабораторная работа. MySQL. Елисейкина Екатерина

Так как недавно вышел новый фильм Марвел, я решила на праздничных днях не думать о лингвистике и взяла датасет с информацией о [фильмах DC и Марвел](https://www.kaggle.com/datasets/leonardopena/marvel-vs-dc)

**Данные и таблицы**


In [4]:
import pandas as pd

In [5]:
path = 'marvel vs dc.csv' #измените путь на свой
df = pd.read_csv(path, encoding='latin1') #пушкинский дом выкладывает файлы в  tsv (tab separated values), поэтому надо указать правильный делиметер
df.head()

,Unnamed: 0,Original Title,Company,Rate,Metascore,Minutes,Release,Budget,Opening Weekend USA,Gross USA,Gross Worldwide
0,1,Iron Man,Marvel,7.9,79,126,2008,140000000,98618668,318604126,585366247
1,2,The Incredible Hulk,Marvel,6.7,61,112,2008,150000000,55414050,134806913,263427551
2,3,Iron Man 2,Marvel,7.0,57,124,2010,200000000,128122480,312433331,623933331
3,4,Thor,Marvel,7.0,57,115,2011,150000000,65723338,181030624,449326618
4,5,Captain America: The First Avenger,Marvel,6.9,66,124,2011,140000000,65058524,176654505,370569774


Лишнее уберем

In [6]:
df = df.drop('Unnamed: 0', axis=1) 

Нечеловеческое перевед в читаемое

In [7]:
df = df.rename(columns={'Gross\xa0USA': 'Gross USA'})

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Original Title       39 non-null     object 
 1   Company              39 non-null     object 
 2   Rate                 39 non-null     float64
 3   Metascore            39 non-null     int64  
 4   Minutes              39 non-null     object 
 5   Release              39 non-null     int64  
 6   Budget               39 non-null     object 
 7   Opening Weekend USA  39 non-null     int64  
 8   Gross USA            39 non-null     int64  
 9   Gross Worldwide      39 non-null     int64  
dtypes: float64(1), int64(5), object(4)
memory usage: 3.2+ KB


Итак, у нас 10 столбцов, которые, на мой взгляд, можно логично разбить на 4 таблички (на схеме ключи и атрибуты отражены)

In [9]:
%pip install mysql-connector-python

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
with open('password.txt', 'r') as f:
    password = f.read().strip()

In [11]:
import mysql.connector
from getpass import getpass

In [12]:
mydb = mysql.connector.connect(
    host = 'localhost',
    user = "root",
    password = password
)

mycursor = mydb.cursor()
mycursor.execute("SHOW DATABASES")

for database in mycursor:
  print(database)

('backoffice',)
('information_schema',)
('lingua',)
('mcu_vs_dcu',)
('mysql',)
('new_schema',)
('performance_schema',)
('sakila',)
('sys',)
('world',)
('writers',)


In [ ]:
mydb = mysql.connector.connect(
  host = "localhost",
  user = "root",
  password = password   
)

mycursor = mydb.cursor()
mycursor.execute("CREATE DATABASE mcu_vs_dcu")

In [14]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = password,
    database = 'mcu_vs_dcu'
)

mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES")

for table in mycursor:
  print(table)

('analytics',)
('companies',)
('finance',)
('movies',)
('ratings',)


Первая табличка простая, но очень нужна будет для аналитики. Два столбца: ключ и имя (их всего два Marvel и DC)

In [35]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = password,
    database = 'mcu_vs_dcu'
)

mycursor = mydb.cursor()
mycursor.execute("""CREATE TABLE companies (
    company_id INT PRIMARY KEY AUTO_INCREMENT,
    company_name VARCHAR(255) NOT NULL UNIQUE
)""")

In [36]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = password,
    database = 'mcu_vs_dcu'
)

mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES")

for table in mycursor:
  print(table)

('companies',)


Получим наименования

In [40]:
companies = df['Company'].unique()
companies = [(company,) for company in companies]
companies

[('Marvel',), ('DC',)]

И запишем их в таблицу

In [41]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = password,
    database = 'mcu_vs_dcu'
)

mycursor = mydb.cursor()
sql = "INSERT INTO companies (company_name) VALUES (%s)"

mycursor.executemany(sql, companies) 
mydb.commit() #не забудьте делать коммит, иначе не сохранится запись в БД

Следующая таблица про фильмы, в которую я включила информацию: название, год выпуска, длительность и айди комании (внешний ключ)

In [42]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = password,
    database = 'mcu_vs_dcu'
)

mycursor = mydb.cursor()
mycursor.execute("""CREATE TABLE movies (
    movie_id INT PRIMARY KEY AUTO_INCREMENT,
    original_title VARCHAR(255) NOT NULL,
    release_year INT,
    duration_min INT,
    company_id INT,
    FOREIGN KEY (company_id) REFERENCES companies(company_id)
)""")

Получим айди компании и остальную информацию для заполнения

In [46]:
mycursor.execute("SELECT company_id, company_name FROM companies")
company_ids = {name: id for id, name in mycursor}
company_ids

{'DC': 2, 'Marvel': 1}

In [50]:
movies = [
    (
        str(df['Original Title'][i]), 
        int(df['Release'][i]),
        int(df['Minutes'][i]), 
        company_ids[df['Company'][i]],
        ) 
        for i in df.index
    ]

movies

[('Iron Man', 2008, 126, 1),
 ('The Incredible Hulk', 2008, 112, 1),
 ('Iron Man 2', 2010, 124, 1),
 ('Thor', 2011, 115, 1),
 ('Captain America: The First Avenger', 2011, 124, 1),
 ('The Avengers', 2012, 143, 1),
 ('Iron Man Three', 2013, 130, 1),
 ('Thor: The Dark World', 2013, 112, 1),
 ('Captain America: The Winter Soldier', 2014, 136, 1),
 ('Guardians of the Galaxy', 2014, 121, 1),
 ('Avengers: Age of Ultron\xa0', 2015, 141, 1),
 ('Ant-Man', 2015, 117, 1),
 ('Captain America: Civil War', 2016, 147, 1),
 ('Doctor Strange', 2016, 115, 1),
 ('Guardians of the Galaxy Vol. 2', 2017, 136, 1),
 ('Spider-Man: Homecoming', 2017, 133, 1),
 ('Thor:Ragnarok', 2017, 130, 1),
 ('Black Panther', 2018, 134, 1),
 ('Avengers: Infinity War', 2018, 149, 1),
 ('Ant-Man and the Wasp', 2018, 118, 1),
 ('Captain Marve', 2019, 123, 1),
 ('Avengers: Endgame', 2019, 181, 1),
 ('Spider-Man: Far from Home', 2019, 129, 1),
 ('Catwoman', 2004, 104, 2),
 ('Batman Begins', 2005, 140, 2),
 ('Superman Returns', 2006

In [52]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = password,
    database = 'mcu_vs_dcu'
)

mycursor = mydb.cursor()
sql = "INSERT INTO movies (original_title, release_year, duration_min, company_id) VALUES (%s, %s, %s, %s)"

mycursor.executemany(sql, movies) 
mydb.commit() 

Таблица с рейтингом и рейтингом кинокритиков

In [53]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = password,
    database = 'mcu_vs_dcu'
)

mycursor = mydb.cursor()
mycursor.execute("""CREATE TABLE ratings (
    movie_id INT NOT NULL,
    rate FLOAT,
    metascore INT,
    FOREIGN KEY (movie_id) REFERENCES movies(movie_id)
)""")

In [54]:
mycursor.execute("SELECT movie_id, original_title FROM movies")
movie_ids = {name: id for id, name in mycursor}
movie_ids

{'Iron Man': 1,
 'The Incredible Hulk': 2,
 'Iron Man 2': 3,
 'Thor': 4,
 'Captain America: The First Avenger': 5,
 'The Avengers': 6,
 'Iron Man Three': 7,
 'Thor: The Dark World': 8,
 'Captain America: The Winter Soldier': 9,
 'Guardians of the Galaxy': 10,
 'Avengers: Age of Ultron\xa0': 11,
 'Ant-Man': 12,
 'Captain America: Civil War': 13,
 'Doctor Strange': 14,
 'Guardians of the Galaxy Vol. 2': 15,
 'Spider-Man: Homecoming': 16,
 'Thor:Ragnarok': 17,
 'Black Panther': 18,
 'Avengers: Infinity War': 19,
 'Ant-Man and the Wasp': 20,
 'Captain Marve': 21,
 'Avengers: Endgame': 22,
 'Spider-Man: Far from Home': 23,
 'Catwoman': 24,
 'Batman Begins': 25,
 'Superman Returns': 26,
 'The Dark Knight': 27,
 'Watchmen': 28,
 'Jonah Hex\xa0': 29,
 'Green Lantern': 30,
 'The Dark Knight Rises': 31,
 'Man of Steel': 32,
 'Batman v Superman: Dawn of Justice': 33,
 'Suicide Squad\xa0': 34,
 'Wonder Woman': 35,
 'Justice League': 36,
 'Aquaman': 37,
 'Shazam!': 38,
 'Joker': 39}

In [56]:
rate = [
    (
        movie_ids[df['Original Title'][i]],
        float(df['Rate'][i]),
        int(df['Metascore'][i]))
        for i in df.index
    ]

rate

[(1, 7.9, 79),
 (2, 6.7, 61),
 (3, 7.0, 57),
 (4, 7.0, 57),
 (5, 6.9, 66),
 (6, 8.0, 69),
 (7, 7.2, 62),
 (8, 6.9, 54),
 (9, 7.7, 70),
 (10, 8.0, 76),
 (11, 7.3, 66),
 (12, 7.3, 64),
 (13, 7.8, 75),
 (14, 7.5, 72),
 (15, 7.6, 67),
 (16, 7.4, 73),
 (17, 7.9, 74),
 (18, 7.3, 88),
 (19, 8.5, 68),
 (20, 7.1, 70),
 (21, 6.9, 64),
 (22, 8.5, 78),
 (23, 7.6, 69),
 (24, 3.3, 27),
 (25, 8.2, 70),
 (26, 6.0, 72),
 (27, 9.0, 84),
 (28, 7.6, 56),
 (29, 4.7, 33),
 (30, 5.5, 39),
 (31, 8.4, 78),
 (32, 7.1, 55),
 (33, 6.5, 44),
 (34, 6.0, 40),
 (35, 7.4, 76),
 (36, 6.4, 45),
 (37, 7.0, 55),
 (38, 7.1, 71),
 (39, 8.7, 59)]

In [57]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = password,
    database = 'mcu_vs_dcu'
)

mycursor = mydb.cursor()
sql = "INSERT INTO ratings (movie_id, rate, metascore) VALUES (%s, %s, %s)"

mycursor.executemany(sql, rate) 
mydb.commit() 

И отдельная таблица про деньги. Тоже нужна для аналитики

In [58]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = password,
    database = 'mcu_vs_dcu'
)

mycursor = mydb.cursor()
mycursor.execute("""CREATE TABLE finance (
    movie_id INT NOT NULL,
    budget DECIMAL(15,2),
    opening_weekend_usa DECIMAL(15,2),
    gross_usa DECIMAL(15,2),
    gross_worldwide DECIMAL(15,2),
    FOREIGN KEY (movie_id) REFERENCES movies(movie_id)
)""")

In [66]:
finance = [
    (
        movie_ids[df['Original Title'][i]],
        int(df['Budget'][i]),
        int(df['Opening Weekend USA'][i]),
        int(df['Gross USA'][i]),
        int(df['Gross Worldwide'][i])
        )
        for i in df.index
    ]
finance

[(1, 140000000, 98618668, 318604126, 585366247),
 (2, 150000000, 55414050, 134806913, 263427551),
 (3, 200000000, 128122480, 312433331, 623933331),
 (4, 150000000, 65723338, 181030624, 449326618),
 (5, 140000000, 65058524, 176654505, 370569774),
 (6, 220000000, 207438708, 623357910, 1518812988),
 (7, 200000000, 174144585, 409013994, 1214811252),
 (8, 170000000, 85737841, 206362140, 644783140),
 (9, 170000000, 95023721, 259766572, 714421503),
 (10, 170000000, 94320883, 333176600, 772776600),
 (11, 250000000, 191271109, 459005868, 1402805868),
 (12, 130000000, 57225526, 180202163, 519311965),
 (13, 250000000, 179139142, 408084349, 1153296293),
 (14, 165000000, 85058311, 232641920, 677718395),
 (15, 200000000, 146510104, 389813101, 863756051),
 (16, 175000000, 117027503, 334201140, 880166924),
 (17, 180000000, 122744989, 315058289, 853977126),
 (18, 200000000, 202003951, 700059566, 1346913161),
 (19, 321000000, 257698183, 678815482, 2048359754),
 (20, 162000000, 75812205, 216648740, 62267

In [67]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = password,
    database = 'mcu_vs_dcu'
)

mycursor = mydb.cursor()
sql = "INSERT INTO finance (movie_id, budget, opening_weekend_usa, gross_usa, gross_worldwide) VALUES (%s, %s, %s, %s, %s)"

mycursor.executemany(sql, finance) 
mydb.commit() 

In [68]:
mycursor = mydb.cursor()
mycursor.execute("SHOW TABLES")

for table in mycursor:
  print(table)

('companies',)
('finance',)
('movies',)
('ratings',)


**Аналитика**

Киновселенная Марвел начинает свой отсчет с 2008 года. Посмотрим, какие фильмы выпустили до этого года

In [28]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = password,
    database = 'mcu_vs_dcu'
)

mycursor = mydb.cursor()
mycursor.execute("""
    SELECT original_title, release_year
FROM movies
WHERE release_year < 2008;
""")
for i in mycursor:
    print(i)



('Catwoman', 2004)
('Batman Begins', 2005)
('Superman Returns', 2006)


Известно, что фильмы про супергероев достаточно длинные. Сколько фильмов из 39 длятся дольше 2.5ч?

In [29]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = password,
    database = 'mcu_vs_dcu'
)

mycursor = mydb.cursor()
mycursor.execute("""
    SELECT Count(
original_title)
FROM movies 
Where duration_min > 150
ORDER BY duration_min DESC
""")
for i in mycursor:
    print(i)

(6,)


Посмотрим на Топ-5 самых прибыльных фильмов.
Логика запроса:
1. Сначала получим данные для расчета прибыли из таблицы finance и названия фильмов с их id из movies (для удобства используем JOIN, хотя можно было бы и вложенный запрос написать, но как я поняла, JOIN работает быстреее, пробуем его)
2. посчитаем доход для каждого фильма и подготовим данные для записи в новую таблицу 
3. сделаем запрос к новой таблице, который покажет фильмы и их доходы в порядке убывания

In [18]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = password,
    database = 'mcu_vs_dcu'
)

mycursor = mydb.cursor()
data = []
mycursor.execute("""
    SELECT m.movie_id, m.original_title, f.budget, f.gross_worldwide 
    FROM movies m
    JOIN finance f ON m.movie_id = f.movie_id
""")
for (movie_id, title, budget, gross) in mycursor:
    roi = ((gross - budget) / budget) * 100
    data.append((movie_id, round(roi, 2)))

data

[(1, Decimal('318.12')),
 (2, Decimal('75.62')),
 (3, Decimal('211.97')),
 (4, Decimal('199.55')),
 (5, Decimal('164.69')),
 (6, Decimal('590.37')),
 (7, Decimal('507.41')),
 (8, Decimal('279.28')),
 (9, Decimal('320.25')),
 (10, Decimal('354.57')),
 (11, Decimal('461.12')),
 (12, Decimal('299.47')),
 (13, Decimal('361.32')),
 (14, Decimal('310.74')),
 (15, Decimal('331.88')),
 (16, Decimal('402.95')),
 (17, Decimal('374.43')),
 (18, Decimal('573.46')),
 (19, Decimal('538.12')),
 (20, Decimal('284.37')),
 (21, Decimal('544.73')),
 (22, Decimal('685.90')),
 (23, Decimal('607.45')),
 (24, Decimal('-17.90')),
 (25, Decimal('148.94')),
 (26, Decimal('44.84')),
 (27, Decimal('443.21')),
 (28, Decimal('42.51')),
 (29, Decimal('-76.80')),
 (30, Decimal('9.93')),
 (31, Decimal('332.42')),
 (32, Decimal('196.91')),
 (33, Decimal('249.45')),
 (34, Decimal('326.77')),
 (35, Decimal('451.58')),
 (36, Decimal('119.31')),
 (37, Decimal('617.60')),
 (38, Decimal('264.57')),
 (39, Decimal('1828.19'))]

In [ ]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = password,
    database = 'mcu_vs_dcu'
)

mycursor = mydb.cursor()
mycursor.execute("""CREATE TABLE analytics (
    movie_id INT NOT NULL,
    roi_value DECIMAL(10,2) NOT NULL,
    FOREIGN KEY (movie_id) REFERENCES movies(movie_id)
)""")

In [19]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = password,
    database = 'mcu_vs_dcu'
)

mycursor = mydb.cursor()
sql = "INSERT INTO analytics (movie_id, roi_value) VALUES (%s, %s)"

mycursor.executemany(sql, data) 
mydb.commit() 

Запрос тоже сделала через JOIN, мне понравилось, это удобнее, чем использовать кучу вложенных селектов

In [22]:
sql = """
SELECT m.original_title, a.roi_value
FROM analytics a
JOIN movies m ON a.movie_id = m.movie_id
ORDER BY roi_value DESC;
"""
mycursor.execute(sql)
myresult = mycursor.fetchall()
for tupl in myresult[:5]:
   print(tupl)

('Joker', Decimal('1828.19'))
('Avengers: Endgame', Decimal('685.90'))
('Aquaman', Decimal('617.60'))
('Spider-Man: Far from Home', Decimal('607.45'))
('The Avengers', Decimal('590.37'))


Интересно, я думала Мстители прибыльнее...

Сейчас много вложенных селектов, чтобы сломать себе весь мозг, но добиться результата (Кстати именно здесь у меня умер kernel, поэтому еще плюс балл JOIN)

Теперь посмотрим на средний рейтинг фильмов каждой киновселенной через вложенный запрос
1. Сначала пишем подзапрос, который выбирает все фильмы одной компании (SELECT movie_id FROM movies WHERE company_id = c.company_id)
2. Вычисляем средний рейтинг по фильмам одной компании (SELECT уровнем выше)
3. Выбираем наименования компании их уже посчитанные в подзапросах рейтинги,сортируя по рейтингу в порядке убывания


In [24]:
sql = """
SELECT c.company_name,
    (SELECT ROUND(AVG(r.rate), 2) 
     FROM ratings r 
     WHERE r.movie_id IN (SELECT movie_id FROM movies WHERE company_id = c.company_id)
    ) AS avg_rating
FROM companies c
ORDER BY avg_rating DESC;
"""
mycursor.execute(sql)
myresult = mycursor.fetchall()
for tupl in myresult:
   print(tupl)

('Marvel', 7.48)
('DC', 6.81)


На самом деле, результат ожидаемый

И в конце посмотрим на лучшие фильмы среди кинокритиков и аудитории

In [33]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = password,
    database = 'mcu_vs_dcu'
)

mycursor = mydb.cursor()
mycursor.execute("""
SELECT original_title, metascore
FROM movies m
JOIN ratings r ON m.movie_id = r.movie_id
ORDER BY r.metascore DESC
""")
myresult = mycursor.fetchall()
for tupl in myresult[:10]:
   print(tupl)

('Black Panther', 88)
('The Dark Knight', 84)
('Iron Man', 79)
('Avengers: Endgame', 78)
('The Dark Knight Rises', 78)
('Wonder Woman', 76)
('Guardians of the Galaxy', 76)
('Captain America: Civil War', 75)
('Thor:Ragnarok', 74)
('Spider-Man: Homecoming', 73)


In [34]:
mydb = mysql.connector.connect(
    host = "localhost",
    user = "root",
    password = password,
    database = 'mcu_vs_dcu'
)

mycursor = mydb.cursor()
mycursor.execute("""
SELECT original_title, rate
FROM movies m
JOIN ratings r ON m.movie_id = r.movie_id
ORDER BY r.rate DESC
""")
myresult = mycursor.fetchall()
for tupl in myresult[:10]:
   print(tupl)

('The Dark Knight', 9.0)
('Joker', 8.7)
('Avengers: Infinity War', 8.5)
('Avengers: Endgame', 8.5)
('The Dark Knight Rises', 8.4)
('Batman Begins', 8.2)
('Guardians of the Galaxy', 8.0)
('The Avengers', 8.0)
('Thor:Ragnarok', 7.9)
('Iron Man', 7.9)


Благодаря этим двух запросам мы видим, что мнение кинокритиков и обычных людей достаточно отличаются. Например, люди выбирают Джокера, в то время, как кинокритики Черную Пантеру. Но любимчиком у всех остается Черный Рыцарь